In [50]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy.sparse as sp

from scipy.sparse import *

from spektral.layers import GCNConv

from keras.layers import Input,Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from keras import Model
from keras.optimizers import Adam
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [51]:
def graph_reader(path):
    """
    Function to read the graph from the path.
    :param path: Path to the edge list.
    :return graph: NetworkX object returned.
    """
    graph = nx.from_edgelist(pd.read_csv(path).values.tolist())
    return graph

def feature_reader(path):
    """
    Reading the sparse feature matrix stored as csv from the disk.
    :param path: Path to the csv file.
    :return features: Dense matrix of features.
    """
    features = pd.read_csv(path)
    node_index = features["node_id"].values.tolist()
    feature_index = features["feature_id"].values.tolist()
    feature_values = features["value"].values.tolist()
    node_count = max(node_index)+1
    feature_count = max(feature_index)+1
    features = coo_matrix((feature_values, (node_index, feature_index)), shape=(node_count, feature_count)).toarray()
    return features

def target_reader(path):
    """
    Reading the target vector from disk.
    :param path: Path to the target.
    :return target: Target vector.
    """
    target = np.array(pd.read_csv(path)["target"]).reshape(-1,1)
    return target

In [52]:
G = graph_reader('data/edges.csv')
X = feature_reader('data/features.csv')
y = target_reader('data/target.csv')

In [53]:
#obtain the adjacency matrix (A)
A = nx.adjacency_matrix(G)
print('Graph info: ', nx.info(G))

/var/folders/jb/995qt8yn7pb1m4_qv3tw0g040000gr/T/ipykernel_22448/2819385825.py:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


Graph info:  Graph with 14746 nodes and 236222 edges


/var/folders/jb/995qt8yn7pb1m4_qv3tw0g040000gr/T/ipykernel_22448/2819385825.py:3: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('Graph info: ', nx.info(G))


In [54]:
X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)

X shape:  (14746, 4)


In [55]:
channels = 16           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping
num_classes =  20       # Number of classes

# Preprocessing operations
A = GCNConv.preprocess(A).astype('f4')

# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GCNConv(channels,
                         activation='relu',
                         kernel_regularizer=regularizers.l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GCNConv(num_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dropout_12 (Dropout)           (None, 4)            0           ['input_13[0][0]']               
                                                                                                  
 input_14 (InputLayer)          [(None, 14746)]      0           []                               
                                                                                                  
 gcn_conv_7 (GCNConv)           (None, 16)           64          ['dropout_12[0][0]',             
                                                                  'input_14[0][0]']         

In [56]:
A.shape

(14746, 14746)

In [57]:
X.shape

(14746, 4)

In [58]:
#Split the data in train and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
A_train, A_test, _, _ = train_test_split(A, y, test_size=0.33, random_state=42)

A_train.shape

(9879, 14746)

In [59]:

# Train model
validation_data = ([X_test,A_test], y_test)

model.fit([X_train,A_train],
          y_train,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=EarlyStopping(patience=10,  restore_best_weights=True))

Epoch 1/200


ValueError: in user code:

    File "/opt/miniconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/opt/miniconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/miniconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/opt/miniconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/miniconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "/opt/miniconda3/envs/ML/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_val